# --- Day 5: Alchemical Reduction ---
You've managed to sneak in to the prototype suit manufacturing lab. The Elves are making decent progress, but are still struggling with the suit's size reduction capabilities.

While the very latest in 1518 alchemical technology might have solved their problem eventually, you can do better. You scan the chemical composition of the suit's material and discover that it is formed by extremely long polymers (one of which is available as your puzzle input).

The polymer is formed by smaller units which, when triggered, react with each other such that two adjacent units of the same type and opposite polarity are destroyed. Units' types are represented by letters; units' polarity is represented by capitalization. For instance, r and R are units with the same type but opposite polarity, whereas r and s are entirely different types and do not react.

For example:
```
In aA, a and A react, leaving nothing behind.
In abBA, bB destroys itself, leaving aA. As above, this then destroys itself, leaving nothing.
In abAB, no two adjacent units are of the same type, and so nothing happens.
In aabAAB, even though aa and AA are of the same type, their polarities match, and so nothing happens.
Now, consider a larger example, dabAcCaCBAcCcaDA:

dabAcCaCBAcCcaDA  The first 'cC' is removed.
dabAaCBAcCcaDA    This creates 'Aa', which is removed.
dabCBAcCcaDA      Either 'cC' or 'Cc' are removed (the result is the same).
dabCBAcaDA        No further actions can be taken.
After all possible reactions, the resulting polymer contains 10 units.
```
How many units remain after fully reacting the polymer you scanned? (Note: in this puzzle and others, the input is large; if you copy/paste your input, make sure you get the whole thing.)

In [1]:
test1 = 'dabAcCaCBAcCcaDA'
filein = 'input5.txt'
with open(filein) as f:
    data = f.read().strip()
f.close()

In [2]:
# polymer = test1
polymer = data

In [3]:
import re
from itertools import cycle,count,compress

def checkPair(ch1,ch2):
    ret = False
    lower = re.compile('[a-z]')
    upper = re.compile('[A-Z]')
    if lower.match(ch1) and upper.match(ch2):
        if ch1 == ch2.lower():
            ret = True
    elif upper.match(ch1) and lower.match(ch2):
        if ch1.lower() == ch2:
            ret = True
    return ret

def clearReactions(polymer,key=None):
    if key == 'even':
        offset = 0
    else:
        offset = 1
        
    
    reactPairs = 0
    n = len(polymer)
    filt = [True]*n
    
    for i in count(0+offset,2):
        if i > n-2-offset:
            break
        elif checkPair(polymer[i],polymer[i+1]):
            reactPairs += 1
            filt[i] = False
            filt[i+1] = False
    
    pl = list(compress(polymer,filt))
    return (pl,reactPairs)

In [4]:
# main
def reactPolymer0(polymer):
    
    plist = list(polymer)

    doneCount = 0
    for key in cycle(['even','odd']):
        passCount = 0 
        plist,rnum = clearReactions(plist,key=key)
        while rnum > 0:
            passCount += 1
            plist,rnum = clearReactions(plist,key=key)

        if passCount == 0:
            doneCount += 1
        else:
            doneCount = 0

        if doneCount == 3:
            break
        
    return ''.join(plist)

def reactPolymer1(polymer):
    changed = True
    while changed:
        i = 0
        changed = False
        while i < len(polymer)-1:
            if checkPair(polymer[i],polymer[i+1]):
                polymer = re.sub(polymer[i]+polymer[i+1],'',polymer)
                changed = True
            else:
                i += 1
#             print(i,":",polymer)
    return polymer



In [5]:
# stolen regex ... much nicer idea than all the looping. 
#Negative lookahead and non capturing group case-insenstitive matching previous group
#([a-zA-Z])(?!\1)(?i:\1)

def reactPolymer(polymer):
    newPolymer = None

    while (newPolymer != polymer):
        newPolymer = polymer
        polymer = re.sub(r'([a-zA-Z])(?!\1)(?i:\1)', '', polymer).strip()
    
    return polymer

from timeit import default_timer as timer

startTime = timer()
print('Part 1(fancy regex): ' + str(len(reactPolymer(polymer))))
endTime = timer()
print('Elapsed Time: ' + str(endTime - startTime))
print()

startTime = timer()
print('Part 1(vers 0): ' + str(len(reactPolymer0(polymer))))
endTime = timer()
print('Elapsed Time: ' + str(endTime - startTime))
print()

startTime = timer()
print('Part 1(vers 1): ' + str(len(reactPolymer1(polymer))))
endTime = timer()
print('Elapsed Time: ' + str(endTime - startTime))
print()


Part 1(fancy regex): 10766
Elapsed Time: 0.3868663791902729

Part 1(vers 0): 10766
Elapsed Time: 14.449888802419512

Part 1(vers 1): 10766
Elapsed Time: 14.030688910179084



# --- Part Two ---
Time to improve the polymer.

One of the unit types is causing problems; it's preventing the polymer from collapsing as much as it should. Your goal is to figure out which unit type is causing the most problems, remove all instances of it (regardless of polarity), fully react the remaining polymer, and measure its length.

For example, again using the polymer dabAcCaCBAcCcaDA from above:
```
Removing all A/a units produces dbcCCBcCcD. Fully reacting this polymer produces dbCBcD, which has length 6.
Removing all B/b units produces daAcCaCAcCcaDA. Fully reacting this polymer produces daCAcaDA, which has length 8.
Removing all C/c units produces dabAaBAaDA. Fully reacting this polymer produces daDA, which has length 4.
Removing all D/d units produces abAcCaCBAcCcaA. Fully reacting this polymer produces abCBAc, which has length 6.
In this example, removing all C/c units was best, producing the answer 4.
```
What is the length of the shortest polymer you can produce by removing all units of exactly one type and fully reacting the result?

In [6]:
alpha = "abcdefghijklmnopqrstuvwxyz"
res = dict()
n = 0
shortest = len(reactPolymer(polymer))
for c in alpha:
    poly = re.sub(c,"",polymer,flags=re.IGNORECASE)
    reactedPoly = reactPolymer(poly)
    if len(reactedPoly) < shortest:
        shortest = len(reactedPoly)
    n += 1
    print(n)
print('shortest: {s}'.format(s=shortest))

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
shortest: 6538
